In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics
import ast

In [ ]:
# вернуть после прогона графиков
#INPUT_DIR = '../input/sf-booking/'
#train_data = pd.read_csv(INPUT_DIR + 'hotels_train.csv')
#test_data = pd.read_csv(INPUT_DIR + 'hotels_test.csv')
#submission = pd.read_csv(INPUT_DIR + 'submission.csv')

In [3]:
hotels = pd.read_csv('data/hotels.csv')

In [4]:
hotels['review_year'] = pd.to_datetime(hotels['review_date']).dt.year
hotels['review_quarter'] = pd.to_datetime(hotels['review_date']).dt.quarter

In [5]:
columns_with_null = (hotels.isnull().mean()*100).sort_values(ascending=False)
columns_with_null

lng                                           0.63288
lat                                           0.63288
hotel_address                                 0.00000
review_total_positive_word_counts             0.00000
review_year                                   0.00000
days_since_review                             0.00000
tags                                          0.00000
reviewer_score                                0.00000
total_number_of_reviews_reviewer_has_given    0.00000
positive_review                               0.00000
additional_number_of_scoring                  0.00000
total_number_of_reviews                       0.00000
review_total_negative_word_counts             0.00000
negative_review                               0.00000
reviewer_nationality                          0.00000
hotel_name                                    0.00000
average_score                                 0.00000
review_date                                   0.00000
review_quarter              

In [6]:
duplicateRows = hotels[hotels.duplicated ()]

In [7]:
hotels = hotels.drop_duplicates()
mask = hotels.duplicated()
hotels_duplicates = hotels[mask]


In [8]:
hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: int(x.split(' ')[0]))

In [9]:
top25_countries = hotels['reviewer_nationality'].value_counts()[:25]

In [10]:
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x if x in top25_countries.index else 'other')

In [11]:
Europe = ['United Kingdom', 'France', 'Switzerland', 'Sweden', 'Turkey', 'Netherlands', 'Italy', 'Ireland', 'Germany','Israel','Spain', 'Romania', 'Belgium', 'Russia', 'Greece']
Asia = ['China', 'India']
USA_and_South_America = ['United States of America', 'Canada']
Middle_East = ['United Arab Emirates', 'Saudi Arabia', 'Kuwait']
Africa = ['South Africa']
Australia_Oceania_Carribbean = ['Australia', 'New Zealand']

In [12]:
hotels['Europe'] = hotels['reviewer_nationality'].apply(
    lambda x: 1 if (Europe[0] in x or
                    Europe[1] in x or
                    Europe[2] in x or
                    Europe[3] in x or
                    Europe[4] in x or
                    Europe[5] in x or
                    Europe[6] in x or
                    Europe[7] in x or
                    Europe[8] in x or
                    Europe[9] in x or
                    Europe[10] in x or
                    Europe[11] in x or
                    Europe[12] in x or
                    Europe[13] in x or
                    Europe[14] in x)  else 0)

hotels['Asia'] = hotels['reviewer_nationality'].apply(
    lambda x: 1 if (Asia[0] in x or
                    Asia[1] in x) else 0)

hotels['USA_and_South_America'] = hotels['reviewer_nationality'].apply(
    lambda x: 1 if (USA_and_South_America[0] in x or
                    USA_and_South_America[1] in x) else 0)

hotels['Middle_East'] = hotels['reviewer_nationality'].apply(
    lambda x: 1 if (Middle_East[0] in x or
                    Middle_East[1] in x or 
                    Middle_East[2] in x) else 0)

hotels['Africa'] = hotels['reviewer_nationality'].apply(
    lambda x: 1 if (Africa[0] in x ) else 0)
        
hotels['Australia_Oceania_Carribbean'] = hotels['reviewer_nationality'].apply(
    lambda x: 1 if (Australia_Oceania_Carribbean[0] in x or
                    Australia_Oceania_Carribbean[1] in x ) else 0)

In [13]:
positive_generally = ['hotel','everything', 'all','location', 'central', 'about', 'metro', 'close', 'station', 'bus', 'airport', 'nearby', 'near', 'center', 'museum', 'boutique', 'centre','distance', 'place', 'theatre', 'position', 'areas' 'proximity', 'situated', 'attractions', 'sights', 'sightseeing', 'locations','view', 'area', 'spa', 'lobby', 'lounge', 'gym', 'windows', 'swimming', 'pool', 'sauna', 'beach', 'hall', 'modern',  'decor', 'design', 'atmosphere',  'interior', 'terrace', 'shuttle', 'lighting', 'light', 'excellent', 'great', 'perfect', 'nice' ]
positive_staff = ['staff', 'friendly', 'helpful', 'service', 'staffs', 'attentive', 'experience', 'accommodating', 'courteous', 'friendliness', 'personnel', 'manager', 'reception', 'receptionist','stuff', 'concierge']
positive_room = ['room', 'equipped','comfortable', 'clean', 'bed', 'rooms', 'bathroom', 'shower', 'comfy', 'wifi', 'beds', 'pillows','bath', 'tv', 'window', 'bedroom', 'towels', 'wi', 'fi', 'internet', 'fridge', 'bathrooms', 'toilet', 'suite', 'kettle', 'linen', 'furniture', 'conditioning', 'toiletries', 'connection', 'confortable', 'cosy', 'cleaned', 'minibar', 'iron', 'charge', 'bedrooms', 'bedding','cleaning', 'cleanliness']
positive_food = ['fresh','delicious', 'breakfast', 'restaurants', 'bar', 'buffet', 'tea', 'food', 'restaurant', 'coffee', 'drinks', 'dinner', 'tasty', 'wine', 'cocktails', 'eggs', 'champagne', 'snacks', 'nespresso', 'kitchen', 'cookies', 'menu', 'meal', 'cake', 'breakfasts', 'chocolate', 'pub', 'dining', 'croissants', 'chocolates', 'juice', 'cocktail']
positive_price = ['price', 'money', 'paid','cheap', 'available']

In [14]:
hotels['positive_generally'] = hotels['positive_review'].apply(
    lambda x: 1 if (positive_generally[0] in x.split(sep=' ') or
                    positive_generally[1] in x.split(sep=' ') or 
                    positive_generally[2] in x.split(sep=' ') or
                    positive_generally[3] in x.split(sep=' ') or
                    positive_generally[4] in x.split(sep=' ') or
                    positive_generally[5] in x.split(sep=' ') or
                    positive_generally[6] in x.split(sep=' ') or
                    positive_generally[7] in x.split(sep=' ') or
                    positive_generally[8] in x.split(sep=' ') or
                    positive_generally[9] in x.split(sep=' ') or
                    positive_generally[10] in x.split(sep=' ') or
                    positive_generally[11] in x.split(sep=' ') or
                    positive_generally[12] in x.split(sep=' ') or
                    positive_generally[13] in x.split(sep=' ') or
                    positive_generally[14] in x.split(sep=' ') or
                    positive_generally[15] in x.split(sep=' ') or
                    positive_generally[16] in x.split(sep=' ') or
                    positive_generally[17] in x.split(sep=' ') or
                    positive_generally[18] in x.split(sep=' ') or
                    positive_generally[19] in x.split(sep=' ') or
                    positive_generally[20] in x.split(sep=' ') or
                    positive_generally[21] in x.split(sep=' ') or
                    positive_generally[22] in x.split(sep=' ') or
                    positive_generally[23] in x.split(sep=' ') or
                    positive_generally[24] in x.split(sep=' ') or
                    positive_generally[25] in x.split(sep=' ') or
                    positive_generally[26] in x.split(sep=' ') or
                    positive_generally[27] in x.split(sep=' ') or
                    positive_generally[28] in x.split(sep=' ') or
                    positive_generally[29] in x.split(sep=' ') or
                    positive_generally[30] in x.split(sep=' ') or
                    positive_generally[31] in x.split(sep=' ') or
                    positive_generally[32] in x.split(sep=' ') or
                    positive_generally[33] in x.split(sep=' ') or
                    positive_generally[34] in x.split(sep=' ') or
                    positive_generally[35] in x.split(sep=' ') or
                    positive_generally[36] in x.split(sep=' ') or
                    positive_generally[37] in x.split(sep=' ') or
                    positive_generally[38] in x.split(sep=' ') or
                    positive_generally[39] in x.split(sep=' ') or
                    positive_generally[40] in x.split(sep=' ') or
                    positive_generally[41] in x.split(sep=' ') or
                    positive_generally[42] in x.split(sep=' ') or
                    positive_generally[43] in x.split(sep=' ') or
                    positive_generally[44] in x.split(sep=' ') or
                    positive_generally[45] in x.split(sep=' ') or
                    positive_generally[46] in x.split(sep=' ') or
                    positive_generally[47] in x.split(sep=' ') or
                    positive_generally[48] in x.split(sep=' ') or
                    positive_generally[49] in x.split(sep=' ') or
                    positive_generally[50] in x.split(sep=' ') or
                    positive_generally[51] in x.split(sep=' ') ) else 0)       
                                        
hotels['positive_staff'] = hotels['positive_review'].apply(
    lambda x: 1 if (positive_staff[0] in x.split(sep=' ') or
                    positive_staff[1] in x.split(sep=' ') or
                    positive_staff[2] in x.split(sep=' ') or
                    positive_staff[3] in x.split(sep=' ') or
                    positive_staff[4] in x.split(sep=' ') or
                    positive_staff[5] in x.split(sep=' ') or
                    positive_staff[6] in x.split(sep=' ') or
                    positive_staff[7] in x.split(sep=' ') or
                    positive_staff[8] in x.split(sep=' ') or
                    positive_staff[9] in x.split(sep=' ') or
                    positive_staff[10] in x.split(sep=' ') or
                    positive_staff[11] in x.split(sep=' ') or
                    positive_staff[12] in x.split(sep=' ') or
                    positive_staff[13] in x.split(sep=' ') or
                    positive_staff[14] in x.split(sep=' ') or
                    positive_staff[15] in x.split(sep=' ') ) else 0)

hotels['positive_room'] = hotels['positive_review'].apply(
    lambda x: 1 if (positive_room[0] in x.split(sep=' ') or 
                    positive_room[1] in x.split(sep=' ') or 
                    positive_room[2] in x.split(sep=' ') or 
                    positive_room[3] in x.split(sep=' ') or 
                    positive_room[4] in x.split(sep=' ') or 
                    positive_room[5] in x.split(sep=' ') or 
                    positive_room[6] in x.split(sep=' ') or 
                    positive_room[7] in x.split(sep=' ') or 
                    positive_room[8] in x.split(sep=' ') or 
                    positive_room[9] in x.split(sep=' ') or 
                    positive_room[10] in x.split(sep=' ') or 
                    positive_room[11] in x.split(sep=' ') or 
                    positive_room[12] in x.split(sep=' ') or 
                    positive_room[13] in x.split(sep=' ') or 
                    positive_room[14] in x.split(sep=' ') or 
                    positive_room[15] in x.split(sep=' ') or 
                    positive_room[16] in x.split(sep=' ') or 
                    positive_room[17] in x.split(sep=' ') or 
                    positive_room[18] in x.split(sep=' ') or 
                    positive_room[19] in x.split(sep=' ') or 
                    positive_room[20] in x.split(sep=' ') or 
                    positive_room[21] in x.split(sep=' ') or 
                    positive_room[22] in x.split(sep=' ') or 
                    positive_room[23] in x.split(sep=' ') or 
                    positive_room[24] in x.split(sep=' ') or 
                    positive_room[25] in x.split(sep=' ') or 
                    positive_room[26] in x.split(sep=' ') or 
                    positive_room[27] in x.split(sep=' ') or 
                    positive_room[28] in x.split(sep=' ') or 
                    positive_room[29] in x.split(sep=' ') or 
                    positive_room[30] in x.split(sep=' ') or 
                    positive_room[31] in x.split(sep=' ') or 
                    positive_room[32] in x.split(sep=' ') or 
                    positive_room[33] in x.split(sep=' ') or 
                    positive_room[34] in x.split(sep=' ') or 
                    positive_room[35] in x.split(sep=' ') or 
                    positive_room[36] in x.split(sep=' ') or 
                    positive_room[37] in x.split(sep=' ') or 
                    positive_room[38] in x.split(sep=' ') or 
                    positive_room[39] in x.split(sep=' ')) else 0)

hotels['positive_food'] = hotels['positive_review'].apply(
    lambda x: 1 if (positive_food[0] in x.split(sep=' ') or
                    positive_food[1] in x.split(sep=' ') or
                    positive_food[2] in x.split(sep=' ') or
                    positive_food[3] in x.split(sep=' ') or
                    positive_food[4] in x.split(sep=' ') or
                    positive_food[5] in x.split(sep=' ') or
                    positive_food[6] in x.split(sep=' ') or
                    positive_food[7] in x.split(sep=' ') or
                    positive_food[8] in x.split(sep=' ') or
                    positive_food[9] in x.split(sep=' ') or
                    positive_food[10] in x.split(sep=' ') or
                    positive_food[11] in x.split(sep=' ') or
                    positive_food[12] in x.split(sep=' ') or
                    positive_food[13] in x.split(sep=' ') or
                    positive_food[14] in x.split(sep=' ') or
                    positive_food[15] in x.split(sep=' ') or
                    positive_food[16] in x.split(sep=' ') or
                    positive_food[17] in x.split(sep=' ') or
                    positive_food[18] in x.split(sep=' ') or
                    positive_food[19] in x.split(sep=' ') or
                    positive_food[20] in x.split(sep=' ') or
                    positive_food[21] in x.split(sep=' ') or
                    positive_food[22] in x.split(sep=' ') or
                    positive_food[23] in x.split(sep=' ') or
                    positive_food[24] in x.split(sep=' ') or
                    positive_food[25] in x.split(sep=' ') or
                    positive_food[26] in x.split(sep=' ') or
                    positive_food[27] in x.split(sep=' ') or
                    positive_food[28] in x.split(sep=' ') or
                    positive_food[29] in x.split(sep=' ') or
                    positive_food[30] in x.split(sep=' ') or
                    positive_food[31] in x.split(sep=' ')) else 0)

hotels['positive_price'] = hotels['positive_review'].apply(
    lambda x: 1 if (positive_price[0] in x.split(sep=' ') or
                    positive_price[1] in x.split(sep=' ') or
                    positive_price[2] in x.split(sep=' ') or
                    positive_price[3] in x.split(sep=' ') or
                    positive_price[4] in x.split(sep=' ')) else 0)

In [15]:
negative_generally = ['negative', 'nothing', 'everything', 'about', 'area', 'location', 'far', 'walk', 'station', 'centre', 'park', 'center', 'metro', 'located', 'areas', 'noisy', 'noise', 'view', 'window', 'facilities', 'parking', 'pool', 'space', 'place', 'windows', 'smell', 'light','lobby', 'loud', 'elevator', 'gym', 'spa','lounge', 'lighting', 'swimming', 'corridor', 'lifts','smoke', 'ceiling','carpets','terrace', 'sauna','smelled', 'smelt', 'elevators', 'hall']
negative_staff = ['staff', 'reception', 'rude', 'experience', 'receptionist', 'manager', 'housekeeping', 'service', 'stuff', 'concierge', 'services', 'unfriendly', 'rude', 'stuffy']
negative_room = ['room', 'bathroom','bed','shower', 'bad', 'rooms', 'wifi', 'beds','bath','toilet', 'size', 'tiny','conditioning', 'uncomfortable', 'charge','bedroom','charged','pillows', 'towels', 'pillow', 'fridge', 'heating', 'sink', 'carpet', 'internet', 'kettle', 'suite', 'wifi', 'tv', 'connection', 'channels','iron', 'curtains', 'internet', 'tube','wi', 'fi', 'conditioner', 'balcony', 'towel', 'wardrobe', 'minibar', 'sheets', 'sofa', 'toiletries','soap','bathrooms', 'ventilation','tub', 'shampoo', 'furniture', 'charges', 'bathtub', 'bedrooms','clean', 'cleaning', 'cleaned', 'clear', 'cleanliness', 'dust', 'dusty', 'dirty', 'rubbish']
negative_food = ['breakfast','bar', 'coffee', 'food', 'tea', 'restaurant', 'drinks', 'menu', 'buffet','milk', 'eggs', 'dinner', 'restaurants', 'meal', 'dining', 'wine','bread','bacon','fruit', 'kitchen', 'egg', 'breakfasts', 'meals', 'bottle', 'drink']
negative_price = ['expensive', 'price', 'cost', 'cheap', 'prices', 'pricey','bill', 'cheaper', 'costs']

In [16]:
hotels['negative_generally'] = hotels['negative_review'].apply(
    lambda x: 1 if (negative_generally[0] in x.split(sep=' ') or
                    negative_generally[1] in x.split(sep=' ') or 
                    negative_generally[2] in x.split(sep=' ') or
                    negative_generally[3] in x.split(sep=' ') or
                    negative_generally[4] in x.split(sep=' ') or
                    negative_generally[5] in x.split(sep=' ') or
                    negative_generally[6] in x.split(sep=' ') or
                    negative_generally[7] in x.split(sep=' ') or
                    negative_generally[8] in x.split(sep=' ') or
                    negative_generally[9] in x.split(sep=' ') or
                    negative_generally[10] in x.split(sep=' ') or
                    negative_generally[11] in x.split(sep=' ') or
                    negative_generally[12] in x.split(sep=' ') or
                    negative_generally[13] in x.split(sep=' ') or
                    negative_generally[14] in x.split(sep=' ') or
                    negative_generally[15] in x.split(sep=' ') or
                    negative_generally[16] in x.split(sep=' ') or
                    negative_generally[17] in x.split(sep=' ') or
                    negative_generally[18] in x.split(sep=' ') or
                    negative_generally[19] in x.split(sep=' ') or
                    negative_generally[20] in x.split(sep=' ') or
                    negative_generally[21] in x.split(sep=' ') or
                    negative_generally[22] in x.split(sep=' ') or
                    negative_generally[23] in x.split(sep=' ') or
                    negative_generally[24] in x.split(sep=' ') or
                    negative_generally[25] in x.split(sep=' ') or
                    negative_generally[26] in x.split(sep=' ') or
                    negative_generally[27] in x.split(sep=' ') or
                    negative_generally[28] in x.split(sep=' ') or
                    negative_generally[29] in x.split(sep=' ') or
                    negative_generally[30] in x.split(sep=' ') or
                    negative_generally[31] in x.split(sep=' ') or
                    negative_generally[32] in x.split(sep=' ') or
                    negative_generally[33] in x.split(sep=' ') or
                    negative_generally[34] in x.split(sep=' ') or
                    negative_generally[35] in x.split(sep=' ') or
                    negative_generally[36] in x.split(sep=' ') or
                    negative_generally[37] in x.split(sep=' ') or
                    negative_generally[38] in x.split(sep=' ') or
                    negative_generally[39] in x.split(sep=' ') or
                    negative_generally[40] in x.split(sep=' ') or
                    negative_generally[41] in x.split(sep=' ') or
                    negative_generally[42] in x.split(sep=' ') or
                    negative_generally[43] in x.split(sep=' ') or
                    negative_generally[44] in x.split(sep=' ') or
                    negative_generally[45] in x.split(sep=' ') ) else 0)       
                                        
hotels['negative_staff'] = hotels['negative_review'].apply(
    lambda x: 1 if (negative_staff[0] in x.split(sep=' ') or
                    negative_staff[1] in x.split(sep=' ') or
                    negative_staff[2] in x.split(sep=' ') or
                    negative_staff[3] in x.split(sep=' ') or
                    negative_staff[4] in x.split(sep=' ') or
                    negative_staff[5] in x.split(sep=' ') or
                    negative_staff[6] in x.split(sep=' ') or
                    negative_staff[7] in x.split(sep=' ') or
                    negative_staff[8] in x.split(sep=' ') or
                    negative_staff[9] in x.split(sep=' ') or
                    negative_staff[10] in x.split(sep=' ') or
                    negative_staff[11] in x.split(sep=' ') or
                    negative_staff[12] in x.split(sep=' ') or
                    negative_staff[13] in x.split(sep=' ')) else 0)

hotels['negative_room'] = hotels['negative_review'].apply(
    lambda x: 1 if (negative_room[0] in x.split(sep=' ') or 
                    negative_room[1] in x.split(sep=' ') or 
                    negative_room[2] in x.split(sep=' ') or 
                    negative_room[3] in x.split(sep=' ') or 
                    negative_room[4] in x.split(sep=' ') or 
                    negative_room[5] in x.split(sep=' ') or 
                    negative_room[6] in x.split(sep=' ') or 
                    negative_room[7] in x.split(sep=' ') or 
                    negative_room[8] in x.split(sep=' ') or 
                    negative_room[9] in x.split(sep=' ') or 
                    negative_room[10] in x.split(sep=' ') or 
                    negative_room[11] in x.split(sep=' ') or 
                    negative_room[12] in x.split(sep=' ') or 
                    negative_room[13] in x.split(sep=' ') or 
                    negative_room[14] in x.split(sep=' ') or 
                    negative_room[15] in x.split(sep=' ') or 
                    negative_room[16] in x.split(sep=' ') or 
                    negative_room[17] in x.split(sep=' ') or 
                    negative_room[18] in x.split(sep=' ') or 
                    negative_room[19] in x.split(sep=' ') or 
                    negative_room[20] in x.split(sep=' ') or 
                    negative_room[21] in x.split(sep=' ') or 
                    negative_room[22] in x.split(sep=' ') or 
                    negative_room[23] in x.split(sep=' ') or 
                    negative_room[24] in x.split(sep=' ') or 
                    negative_room[25] in x.split(sep=' ') or 
                    negative_room[26] in x.split(sep=' ') or 
                    negative_room[27] in x.split(sep=' ') or 
                    negative_room[28] in x.split(sep=' ') or 
                    negative_room[29] in x.split(sep=' ') or 
                    negative_room[30] in x.split(sep=' ') or 
                    negative_room[31] in x.split(sep=' ') or 
                    negative_room[32] in x.split(sep=' ') or 
                    negative_room[33] in x.split(sep=' ') or 
                    negative_room[34] in x.split(sep=' ') or 
                    negative_room[35] in x.split(sep=' ') or 
                    negative_room[36] in x.split(sep=' ') or 
                    negative_room[37] in x.split(sep=' ') or 
                    negative_room[38] in x.split(sep=' ') or 
                    negative_room[39] in x.split(sep=' ') or 
                    negative_room[40] in x.split(sep=' ') or 
                    negative_room[41] in x.split(sep=' ') or 
                    negative_room[42] in x.split(sep=' ') or 
                    negative_room[43] in x.split(sep=' ') or 
                    negative_room[44] in x.split(sep=' ') or 
                    negative_room[45] in x.split(sep=' ') or 
                    negative_room[46] in x.split(sep=' ') or 
                    negative_room[47] in x.split(sep=' ') or 
                    negative_room[48] in x.split(sep=' ') or 
                    negative_room[49] in x.split(sep=' ') or 
                    negative_room[50] in x.split(sep=' ') or 
                    negative_room[51] in x.split(sep=' ') or 
                    negative_room[52] in x.split(sep=' ') or 
                    negative_room[53] in x.split(sep=' ') or 
                    negative_room[54] in x.split(sep=' ') or 
                    negative_room[55] in x.split(sep=' ') or 
                    negative_room[56] in x.split(sep=' ') or 
                    negative_room[57] in x.split(sep=' ') or 
                    negative_room[58] in x.split(sep=' ') or 
                    negative_room[59] in x.split(sep=' ') or 
                    negative_room[60] in x.split(sep=' ') or 
                    negative_room[61] in x.split(sep=' ') or 
                    negative_room[62] in x.split(sep=' ')) else 0)

hotels['negative_food'] = hotels['negative_review'].apply(
    lambda x: 1 if (negative_food[0] in x.split(sep=' ') or
                    negative_food[1] in x.split(sep=' ') or
                    negative_food[2] in x.split(sep=' ') or
                    negative_food[3] in x.split(sep=' ') or
                    negative_food[4] in x.split(sep=' ') or
                    negative_food[5] in x.split(sep=' ') or
                    negative_food[6] in x.split(sep=' ') or
                    negative_food[7] in x.split(sep=' ') or
                    negative_food[8] in x.split(sep=' ') or
                    negative_food[9] in x.split(sep=' ') or
                    negative_food[10] in x.split(sep=' ') or
                    negative_food[11] in x.split(sep=' ') or
                    negative_food[12] in x.split(sep=' ') or
                    negative_food[13] in x.split(sep=' ') or
                    negative_food[14] in x.split(sep=' ') or
                    negative_food[15] in x.split(sep=' ') or
                    negative_food[16] in x.split(sep=' ') or
                    negative_food[17] in x.split(sep=' ') or
                    negative_food[18] in x.split(sep=' ') or
                    negative_food[19] in x.split(sep=' ') or
                    negative_food[20] in x.split(sep=' ') or
                    negative_food[21] in x.split(sep=' ') or
                    negative_food[22] in x.split(sep=' ') or
                    negative_food[23] in x.split(sep=' ') or
                    negative_food[24] in x.split(sep=' ') ) else 0)

hotels['negative_price'] = hotels['negative_review'].apply(
    lambda x: 1 if (negative_price[0] in x.split(sep=' ') or
                    negative_price[1] in x.split(sep=' ') or
                    negative_price[2] in x.split(sep=' ') or
                    negative_price[3] in x.split(sep=' ') or
                    negative_price[4] in x.split(sep=' ') or
                    negative_price[5] in x.split(sep=' ') or
                    negative_price[6] in x.split(sep=' ') or
                    negative_price[7] in x.split(sep=' ') or
                    negative_price[8] in x.split(sep=' ')) else 0)

In [17]:
num_night = [' Stayed 1 night ',' Stayed 2 nights ',' Stayed 3 nights ',' Stayed 4 nights ',' Stayed 5 nights ',' Stayed 6 nights ',\
             ' Stayed 7 nights ',' Stayed 8 nights ',' Stayed 9 nights ',' Stayed 10 nights ',' Stayed 11 nights ',\
             ' Stayed 12 nights ',' Stayed 13 nights ',' Stayed 14 nights ',' Stayed 15 nights ',' Stayed 16 nights ',' Stayed 17 nights ',\
              ' Stayed 18 nights ',' Stayed 19 nights ',' Stayed 20 nights ',' Stayed 21 nights ',' Stayed 22 nights ',' Stayed 23 nights ',\
             ' Stayed 24 nights ',' Stayed 25 nights ',' Stayed 26 nights ',' Stayed 27 nights ',' Stayed 28 nights ',' Stayed 29 nights ',' Stayed 30 nights ']

def get_night(x):
  for i in num_night:
    if i in x:
      return int(i.split(' ')[2])

hotels['nights'] = hotels['tags'].apply(get_night)

In [18]:
hotels['nights'] = hotels['nights'].fillna(1)

In [19]:
hotels['Leisure_trip'] = hotels['tags'].apply(lambda x: 1 if ' Leisure trip ' in x else 0)
hotels['Business_trip'] = hotels['tags'].apply(lambda x: 1 if ' Business trip ' in x else 0)
hotels['with_mobile'] = hotels['tags'].apply(lambda x: 1 if ' Submitted from a mobile device ' in x else 0)


In [20]:
def get_travelers(x):
  if ' Couple ' in x:
    return 'Couple'
  elif ' Solo traveler ' in x:
    return 'Solo traveler'
  elif ' Group ' in x:
    return 'Group'
  elif ' Family with young children ' in x:
    return 'Family with young children'
  elif ' Family with older children ' in x:
    return 'Family with older children'
  elif ' Travelers with friends ' in x:
    return 'Travelers with friends'       

hotels['Travelers_type'] = hotels['tags'].apply(get_travelers)

In [21]:
lst_room = [' Twin Room ',' Double Room ',' Single ',' Queen ',' King Room ',' Triple ',' Suite ',\
            ' Studio ',' Family Room ',' Quadruple ',' 2 rooms ',' 3 rooms ',' Standard Room ',' Classic Room ',\
            ' Twin Guest Room',' Double Hilton Guestroom ',' Standard Room with 1 Double Bed ',' Standard Double ',\
            ' Double Deluxe Room ',' Studio Suite ',' King Hilton Guest Room ']

lst_room1 = [' Single ',' 1 room ']
lst_room2 = [' Twin Room ',' Double Room ',' 2 rooms ',' Standard Room ',' Classic Room ',' Twin Guest Room',\
             ' Double Hilton Guestroom ',' Standard Room with 1 Double Bed ',' Standard Double ']
lst_room3 = [' Triple ',' 3 rooms ']
lst_room4 = [' Quadruple ']
lst_room5 = [' Queen ']
lst_room6 = [' King Room ',' King Hilton Guest Room ']
lst_room7 = [' Suite ']
lst_room8 = [' Studio ',' Studio Suite ']

def get_room(x):
  for i in lst_room:
    if i in lst_room1 and i in x:
      return 'Single'
    if i in lst_room2 and i in x:
      return 'Double_Twin'
    if i in lst_room3 and i in x:
      return 'Triple'
    if i in lst_room4 and i in x:
      return 'Quadruple'
    if i in lst_room5 and i in x:
      return 'Queen'
    if i in lst_room6 and i in x:
      return 'King'
    if i in lst_room7 and i in x:
      return 'Suite'
    if i in lst_room8 and i in x:
      return 'Studio'      

hotels['room_type'] = hotels['tags'].apply(get_room)

In [22]:
hotels['room_type'].isnull().mean()
hotels['room_type'] = hotels['room_type'].fillna('Double_Twin')

In [23]:
hotels.drop(['hotel_address','review_date','hotel_name','reviewer_nationality','negative_review',\
             'positive_review','tags','lat','lng','review_year'], axis=1, inplace=True)

In [24]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386496 entries, 0 to 386802
Data columns (total 31 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                386496 non-null  int64  
 1   average_score                               386496 non-null  float64
 2   review_total_negative_word_counts           386496 non-null  int64  
 3   total_number_of_reviews                     386496 non-null  int64  
 4   review_total_positive_word_counts           386496 non-null  int64  
 5   total_number_of_reviews_reviewer_has_given  386496 non-null  int64  
 6   reviewer_score                              386496 non-null  float64
 7   days_since_review                           386496 non-null  int64  
 8   review_quarter                              386496 non-null  int64  
 9   Europe                                      386496 non-null  int64  
 

In [178]:
hotels = pd.get_dummies(hotels,columns=['review_quarter','room_type','Travelers_type'])

In [179]:
hotels.drop(['additional_number_of_scoring'],axis=1,inplace=True)

In [180]:
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [181]:
hotels.isnull().mean()

average_score                                 0.0
review_total_negative_word_counts             0.0
total_number_of_reviews                       0.0
review_total_positive_word_counts             0.0
total_number_of_reviews_reviewer_has_given    0.0
reviewer_score                                0.0
days_since_review                             0.0
Europe                                        0.0
Asia                                          0.0
USA_and_South_America                         0.0
Middle_East                                   0.0
Africa                                        0.0
Australia_Oceania_Carribbean                  0.0
positive_generally                            0.0
positive_staff                                0.0
positive_room                                 0.0
positive_food                                 0.0
positive_price                                0.0
negative_generally                            0.0
negative_staff                                0.0


In [182]:
from sklearn.model_selection import train_test_split  

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor # the tool for create and train the model 
from sklearn import metrics # the tools to evaluate the accuracy of the model
  
# Create the model 
regr = RandomForestRegressor(n_estimators=100)  
      
# Train the model on the train data
regr.fit(X_train, y_train)  
      
# Use a trained model to predict the rating of hotels in the test sample.  
# The predicted values are written to the y_pred 
y_pred = regr.predict(X_test)  

In [186]:
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.1318021255481249
